In [ ]:
import sys
import os
import os.path
# actual libraries
import re
import logging
import numpy as np
import pandas as pd
import scanpy as sc
import scipy.stats as sps
from anndata import AnnData
import anndata
from collections import defaultdict, OrderedDict
import plotly.express.colors as pxcolors
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# local to this analysis
import de
import plotting
import scoring
import signatures
import util

FORMAT = '%(asctime)-15s %(message)s'
logging.basicConfig(format=FORMAT)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

sc.settings.verbosity = 3
sc.logging.print_header()
sc.settings.set_figure_params(dpi=120, dpi_save=480)

In [ ]:
# Configuration
figure_dir = '../../build/figures'
sc.settings.figdir = figure_dir
supplement_dir = '../../build/supplement'

dataset = 'GSE154659'

model = 'ScNT'
neuron_filter = {
    'model': [model, 'Naive'],
    'mouse': 'C57',
    'subtype': [
        'SST',
        'NP',
        'PEP1',
        'NF2',
        'NF1',
        'NF3',
        'p_cLTMR2',
        'cLTMR1',
        'PEP2',
    ],
    'n_counts': lambda x: x <= 15000,
    'predicted_doublet': False,
}
non_neuron_filter = {
    'model': [model, 'Naive'],
    'mouse': 'C57',
    'subtype': [
        'Schwann',
        'Repair schwann',
        'B cell',
        'Endothelial',
        'Fibroblast',
        'Macrophage',
        'Neutrophil',
        'Repair fibroblast',
        'Satglia',
        'Pericyte',
    ],
    'n_counts': lambda x: x <= 15000,
    'predicted_doublet': False,
}


# abs(log2 fold change) must be greater than this
l2fc_thresh = .6 

# per cell gene signatures to assess
gene_signatures = [
    'Cdkn2a,-Lmnb1,-Top2a',
    'Cdkn1a,Cdkn2a,-Lmnb1,-Top2a',
    'Cdkn1a,-Lmnb1,-Top2a',
    'Cdkn2a,-Top2a',
]

# in some cases we group cells into larger cell type groups
# according to this scheme
cell_groups = dict(
    neuron=[
        'SST',
        'NP',
        'PEP1',
        'NF2',
        'NF1',
        'NF3',
        'p_cLTMR2',
        'cLTMR1',
        'PEP2',
    ],
    glia=[
        'Schwann',
        'Repair schwann',
        'Satglia',
    ],
    other=[
        'B cell',
        'Endothelial',
        'Fibroblast',
        'Macrophage',
        'Neutrophil',
        'Repair fibroblast',
        'Pericyte',
    ],
)



In [ ]:
# Load Dataset 
#client = dev_instance()
#broker = client.get_databroker(dataset)
#adata = broker.load_variant('lin_norm')

adata = sc.read_h5ad(os.path.join('../../build/datasets/', dataset, f'{dataset}.h5ad'))

adata

In [ ]:
adata.obs['predicted_doublet'].value_counts()

In [ ]:
adata.var_names = [x.capitalize() for x in adata.var_names]

In [ ]:
# split dataset into neurons and non-neurons according to filters described above
# filter datasets to only contains the configured model and cell types
adata_neuron = util.adata_filter(adata, neuron_filter).copy()
adata_non_neuron = util.adata_filter(adata, non_neuron_filter).copy()
adata_neuron


In [ ]:
adata_non_neuron

In [ ]:

# add some categorizations here to help group cells
# label: A combination of model and time-point
# label_subtype: a combination of model, time-point, and subtype

adata_neuron.obs['label'] = [f'{model}_{int(hour):04}h' if cmodel == model else cmodel for (cmodel, hour) in zip(adata_neuron.obs['model'], adata_neuron.obs['hour'])]
adata_neuron.obs['label_subtype'] = [f'{subtype}_{int(hour):04}h' if cmodel == model else f'{subtype}_{0:04}_Naive' for (cmodel, subtype, hour) in zip(adata_neuron.obs['model'], adata_neuron.obs['subtype'], adata_neuron.obs['hour'])]

adata_non_neuron.obs['label'] = [f'{model}_{int(hour):04}h' if cmodel == model else model for (cmodel, hour) in zip(adata_non_neuron.obs['model'], adata_non_neuron.obs['hour'])]
adata_non_neuron.obs['label_subtype'] = [f'{subtype}_{int(hour):04}h' if cmodel == model else f'{subtype}_{0:04}_Naive' for (cmodel, subtype, hour) in zip(adata_non_neuron.obs['model'], adata_non_neuron.obs['subtype'], adata_non_neuron.obs['hour'])]

adata_neuron.obs['label'].unique()
adata_neuron.obs['label_subtype'].unique()

In [ ]:
# list of unique labels
adata_neuron.obs['label'].unique()

In [ ]:
# list of unique subtype labels
adata_neuron.obs['label_subtype'].unique()

In [ ]:
# create an additional label that includes whether a cell has nonzero expression of ATF3

def atf3_label(l, a):
    if l == 'Naive':
        return l
    if a:
        return f'{l}_Atf3+'
    else:
        return f'{l}_Atf3-'
adata_neuron.obs['atf3_label'] = [atf3_label(l, a) for (l, a) in zip(adata_neuron.obs['label'], util.adata_filter_mask(adata_neuron, {'Atf3': {'gt': 0.0}}))]
adata_neuron.obs['atf3_label'].unique()

In [ ]:
# create differential expression contrasts
# for each of label, label_subtype, and atf3_label, create a comparison between that grouping of cells, and the comperable grouping of Naive cells
# in the resulting 'comparisons' dictionary, t
# key is: "Forground:Background" where Foreground and Background are the names of the group of cells, for example "ScNT_1440h:Naive"
# value is: a tuple of filter dictionaries (each composed of {"obs_key": "required_value"} as accepted by the adata_filter_mask() function) 
#   one for foreground cells, and one for background

naive = None
naive_subtype = None
modeled = {}
modeled_subtype = {}
modeled_atf3 = {}
for i, row in adata_neuron.obs.iterrows():
    model = row['model']
    hour = row['hour']
    label = row['label']
    label_subtype = row['label_subtype']
    atf3_label = row['atf3_label']

    if model == 'Naive':
        if naive is None:
            naive = {'label': label}
        if naive_subtype is None:
            naive_subtype = {'label_subtype': label_subtype}
    else:
        val = (int(hour), {'label': label})
        if label not in modeled:
            modeled[label] = val
            
        val_subtype = (int(hour), {'label_subtype': label_subtype})
        if label_subtype not in modeled_subtype:
            modeled_subtype[label_subtype] = val_subtype

        val_atf3 = (int(hour), {'atf3_label': atf3_label})
        if atf3_label not in modeled_atf3:
            modeled_atf3[atf3_label] = val_atf3

modeled = [y[1] for y in sorted(modeled.values(), key=lambda x: x[0])]
modeled_atf3 = [y[1] for y in sorted(modeled_atf3.values(), key=lambda x: x[0])]
modeled_subtype = [y[1] for y in sorted(modeled_subtype.values(), key=lambda x: x[0])]

comparisons = {}
for m in modeled:
    v = list(m.values())[0]
    n = list(naive.values())[0]
    comparisons[f'{v}:{n}'] = (m, naive)
for m in modeled_atf3:
    v = list(m.values())[0]
    n = list(naive.values())[0]
    comparisons[f'{v}:{n}'] = (m, naive)
for m in modeled_subtype:
    v = list(m.values())[0]
    n = list(naive_subtype.values())[0]
    comparisons[f'{v}:{n}'] = (m, naive_subtype)

comparisons
    

In [ ]:
# compute differential expression for neurons based on the above configured contrasts
# takes the dictionary of contrast_name to contrast filters
# it returns a dictionary of contrast_name -> pandas.DataFrame
# each DataFrame contains the per-gene differential expression statistics
des_neuron = de.differential_expression(adata_neuron, comparisons)
des_neuron['NF3_1440h:SST_0000_Naive']

In [ ]:
# use the flag_de function to add a boolean 'is-de' column to each DataFrame
# log2fc_thresh: abs(log2fc) must be greater than this
# p_column: which computed statistic should be used as a significance threshold
# p_thresh: the value of the p_column must be <= p_thresh
de.flag_de(des_neuron, log2fc_thresh=l2fc_thresh, p_column='ranksums-fdr-p', p_thresh=.05)

# show an example differential expression table
det = des_neuron['NF3_1440h:SST_0000_Naive']
det[det['is-de']]
    

In [ ]:
# split the differential expression tables up into 3 different analysis groups: label, label_subtype, label_atf3
des_neuron_label = {k: v for k, v in des_neuron.items() if k.endswith(':Naive') and 'Atf3' not in k}
des_neuron_label_subtype = {k: v for k, v in des_neuron.items() if not k.endswith(':Naive') and 'Atf3' not in k}
des_neuron_atf3 = {k: v for k, v in des_neuron.items() if 'Atf3' in k}



# Plot Core Senescence genes, core chronic pain markers, and senescence associated genes from SenMayo and SASP constituants

Core Senescence Genes: CDKN1A, CDKN2A

Core Pain Genes: ATF3, IL6, IL1B


In [ ]:
# plot differentially expressed genes from included gene sets
# shows only genes that are differential expressed in at least one of the differential expression tables
plotting.plot_de_genes(
    adata_neuron, 
    'label', 
    des_neuron_label, 
    genesets={'senmayo': 'senmayo_mouse', 'sasp_review': 'sasp_review_mouse'}, 
    genes=['Cdkn1a', 'Cdkn2a', 'Atf3', 'Il6', 'Il1b'],
    sort_genes_by='expr',
    smallest_dot=8.,
    dot_min=.0,
    save='renthal_senescence.png',
)

In [ ]:
# output also the stats for the above plot
label_de_summary = de.summarize_de_genes(des_neuron_label, genesets={'senmayo': 'senmayo_mouse', 'sasp_review': 'sasp_review_mouse'}, genes=['Cdkn1a', 'Cdkn2a', 'Atf3', 'Il6', 'Il1b'])
label_de_summary.to_excel(os.path.join(supplement_dir, 'renthal_senescence.xlsx'))
label_de_summary

In [ ]:
# for plotting purposes order the atf3 labels Naive, ATF3-, ATF3+, and otherwise by timepoint
atf3_labels = list(adata_neuron.obs['atf3_label'].unique())
def sort_order(atf3_label):
    if atf3_label == 'Naive':
        return 0
    sp = atf3_label.split('_')
    model, hour, atf3 = sp
    hv = int(hour[:-1])
    if atf3 == 'Atf3+':
        hv += 10000
    return hv
atf3_labels.sort(key=sort_order)
atf3_labels

In [ ]:
# plot differentially expressed genes from included gene sets
# shows only genes that are differential expressed in at least one of the differential expression tables
plotting.plot_de_genes(
    adata_neuron, 
    'atf3_label', 
    des_neuron_atf3, 
    genesets={'senmayo': 'senmayo_mouse', 'sasp_review': 'sasp_review_mouse'}, 
    genes=['Cdkn1a', 'Cdkn2a', 'Atf3', 'Il6', 'Il1b'],
    sort_genes_by='expr',
    smallest_dot=8.,
    dot_min=.0,
    categories_order=atf3_labels,
    save='renthal_atf3_split_senescence.png',
)

In [ ]:
# output also the stats for the above plot
atf3_de_summary = de.summarize_de_genes(des_neuron_atf3, genesets={'senmayo': 'senmayo_mouse', 'sasp_review': 'sasp_review_mouse'}, genes=['Cdkn1a', 'Cdkn2a', 'Atf3', 'Il6', 'Il1b'])
atf3_de_summary.to_excel(os.path.join(supplement_dir, 'renthal_atf3_split_senescence.xlsx'))
atf3_de_summary

In [ ]:
# select from the senmayo geneset only the genes that have some kind of differential expression at any model timepoint
senmayo_de_genes = de.get_de_genes(des_neuron, genes=['Cdkn1a', 'Cdkn2a'], genesets={'senmayo': 'senmayo_mouse'})
print(f'SenMayo DE Genes (at any timepoint): {senmayo_de_genes}')

# using this subset of senmayo genes, score each single cell for its expression of genes in the gene-set
# this is normalized within each neuron subtype
scoring.score_within_key(
    adata_neuron, 
    list(senmayo_de_genes),
    'subtype',
    'senmayo_score',
)
    
        
    

In [ ]:
# plot heatmap of senmayo scores across model timepoints and neuron subtypes
fig = plotting.plot_score_heatmap(adata_neuron, 'subtype', 'hour', 'senmayo_score', y_as='int')
fig.update_layout(title='Mean SenMayo Score', font=dict(family='arial', size=34))
pio.write_image(fig, os.path.join(figure_dir, 'renthal_senmayo_neuron_heatmap.svg'), scale=6, width=1080, height=1080)
pio.write_image(fig, os.path.join(figure_dir, 'renthal_senmayo_neuron_heatmap.png'), scale=6, width=1080, height=1080)
fig.show()

In [ ]:
# create a new AnnData object only containing the normalized senmayo scores
adata_neuron_scores = AnnData(adata_neuron.obs[['senmayo_score']])
adata_neuron_scores.obs = adata_neuron.obs
adata_neuron_scores


In [ ]:
# compute "differential expression" of senmayo scores in order to generate statistical significace values for changes in score
des_neuron_scores = de.differential_expression(adata_neuron_scores, comparisons, tests=['ranksums', 'ttest'])

In [ ]:
det = des_neuron_scores['ScNT_0024h:Naive']
det

In [ ]:
# output the score differential expression statistics
rows = []
for c, de in des_neuron_scores.items():
    if c.endswith(':Naive'):
        continue
    sp = c.split(':')
    fg = sp[0]
    bg = sp[1]
    st = '_'.join(fg.split('_')[:-1])
    row = de.iloc[0]
    rp = row['ranksums-p']
    tp = row['ttest-p']
    fga = row['fg_lin-avg']
    bga = row['bg_lin-avg']
    rows.append([c, st, fg, bg, fga, bga, fga - bga, rp, tp, rp < .05, tp < .05])
score_stats = pd.DataFrame(rows, columns=['contrast', 'subtype', 'foreground', 'background', 'foreground_mean', 'background_mean', 'score_delta', 'ranksums_p', 'ttest_p', 'ranksums_significant', 'ttest_significant'])
score_stats = score_stats.sort_values('contrast')
score_stats.set_index('contrast', inplace=True)
score_stats.to_excel(os.path.join(supplement_dir, 'renthal_senmayo_neuron_scores_stats.xlsx'))
score_stats
    
    

In [ ]:
# assess configured gene signatures on all neurons and non-neurons
# data is log1p transformed and scaled to per-gene variance without zero centering

adata_neuron_scaled = adata_neuron.copy()
sc.pp.log1p(adata_neuron_scaled)
sc.pp.scale(adata_neuron_scaled, zero_center=False)
signatures.assess_signatures(adata_neuron_scaled, gene_signatures)

adata_non_neuron_scaled = adata_non_neuron.copy()
sc.pp.log1p(adata_non_neuron_scaled)
sc.pp.scale(adata_non_neuron_scaled, zero_center=False)
signatures.assess_signatures(adata_non_neuron_scaled, gene_signatures)

In [ ]:
# summarize signatures, and clean up the output a little bit for better export
adata_neuron_scaled.obs['compartment'] = 'neuronal'
adata_non_neuron_scaled.obs['compartment'] = 'non-neuronal'
signatures_obs = pd.concat([adata_neuron_scaled.obs, adata_non_neuron_scaled.obs])

# append a cellgroup annotation based on configuration above
def getgrp(x):
    for k, v in cell_groups.items():
        if x in v:
            return k
signatures_obs['cellgroup'] = [getgrp(s) for s in signatures_obs['subtype']]

# add the hour as an integer for sorting purposes
signatures_obs['hour_int'] = [int(x) for x in signatures_obs['hour']]

signatures_df = signatures.summarize_signatures(
    signatures_obs,
    gene_signatures,
    groupby=['hour', 'cellgroup'],
    include_columns=['subtype', 'compartment', 'hour_int'],
    totals_groupby='hour',
    sort_by=['hour_int', 'cellgroup'],
)

# hour_int column not really needed, was just for sorting
signatures_df = signatures_df[[c for c in signatures_df.columns if c != 'hour_int']]
signatures_df.to_excel(os.path.join(supplement_dir, f'renthal_sen_signatures.xlsx'))

signatures_df

In [ ]:
# quick timecourse plot of signature positivity
# these are not exported as the final plots are made in prism from the spreadsheet
def plot_signature_summary(sig_df, column, **layout_kwargs):
    fig = px.line(sig_df, x='hour', y=column, color='cellgroup')
    fig.update_layout(**layout_kwargs)
    return fig

for sig in gene_signatures:
    gp = f'{sig}_percent_group_positive'
    plot_signature_summary(signatures_df, gp, height=800, width=1000, title=f'{sig} Percent of Cell Group').show()
                           
    ap = f'{sig}_percent_all_positive'
    plot_signature_summary(signatures_df, ap, height=800, width=1000, title=f'{sig} Percent of All Cells (per timepoint)').show()



In [ ]:
# output all differentially expressed genes for neuron timepoints, subtypes, and atf3 positivity combinations

with pd.ExcelWriter(os.path.join(supplement_dir, 'renthal_differential_expression.xlsx')) as writer:
    for de_name, de in des_neuron.items():
        sheet_name = re.sub(':', '_vs_', de_name)
        sheet_name = re.sub('_0000', '', sheet_name)
        de.to_excel(writer, sheet_name=sheet_name)